# Westminster Ground Truth Analysis

This notebook demonstrates the complete workflow for creating orthomosaics from DJI drone imagery and evaluating their accuracy:

1. **Data Loading**: Load images, GCPs, and DJI metadata
2. **Feature Detection & Matching**: Detect and match features across images
3. **Camera Pose Estimation**: Estimate initial camera poses
4. **Bundle Adjustment**: Refine poses to minimize reprojection error
5. **Orthomosaic Creation**: Generate orthomosaics with and without GCPs
6. **Basemap Comparison**: Download basemaps and quantify absolute accuracy
7. **Visualization**: Visualize matches, errors, and results


## Setup: Install Dependencies

First, install the required packages if they're not already installed.


In [26]:
# Install required packages
import subprocess
import sys
from pathlib import Path

# Try to install from requirements.txt first
requirements_file = Path("requirements.txt")
if requirements_file.exists():
    print("Installing packages from requirements.txt...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", str(requirements_file)])
    print("✓ Packages installed from requirements.txt")
else:
    # Fallback: install packages individually
    print("requirements.txt not found. Installing packages individually...")
    packages = [
        "numpy>=1.24.0",
        "opencv-python>=4.8.0",
        "scipy>=1.11.0",
        "scikit-image>=0.21.0",
        "rasterio>=1.3.0",
        "pillow>=10.0.0",
        "matplotlib>=3.7.0",
        "pandas>=2.0.0",
        "pyproj>=3.6.0",
        "shapely>=2.0.0",
        "requests>=2.31.0",
        "tqdm>=4.66.0",
        "exifread>=3.0.0",
        "utm>=0.7.0"
    ]
    for package in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    print("✓ All packages installed")

print("\nSetup complete!")


Installing packages from requirements.txt...
✓ Packages installed from requirements.txt

Setup complete!


## Imports


In [27]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Add package to path
sys.path.insert(0, str(Path.cwd()))

from westminster_ground_truth_analysis import (
    GCPParser,
    DJIMetadataParser,
    OrthomosaicPipeline,
    download_basemap,
    compare_orthomosaic_to_basemap,
    visualize_matches,
    visualize_reprojection_errors,
    visualize_camera_poses,
    create_match_quality_report
)

# Set up paths
data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
output_dir = Path("outputs")
output_dir.mkdir(exist_ok=True)

print("Setup complete!")


Setup complete!


## 1. Load Ground Control Points


In [28]:
# Parse GCP file
gcp_file = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
gcp_parser = GCPParser(str(gcp_file))

gcps = gcp_parser.get_gcps()
print(f"Loaded {len(gcps)} ground control points")

# Display first few GCPs
for gcp in gcps[:5]:
    print(f"  {gcp.name}: X={gcp.x:.2f}, Y={gcp.y:.2f}, Z={gcp.z:.2f}")

# Get bounds
min_x, min_y, max_x, max_y = gcp_parser.get_bounds()
print(f"\nGCP Bounds: X=[{min_x:.2f}, {max_x:.2f}], Y=[{min_y:.2f}, {max_y:.2f}]")


Loaded 23 ground control points
  GCP1: X=5450945.53, Y=506914.12, Z=77.45
  GCP2: X=5450730.01, Y=506657.79, Z=79.22
  GCP3: X=5450480.01, Y=506577.77, Z=59.40
  GCP4: X=5450578.63, Y=506765.03, Z=65.59
  GCP5: X=5450715.96, Y=506926.13, Z=63.10

GCP Bounds: X=[5450109.82, 5450992.66], Y=[506577.77, 507315.01]


## 2. Process First Dataset (DJI_202510060955_017_25-3288)


In [29]:
# Setup for first dataset
dataset1_dir = data_dir / "DJI_202510060955_017_25-3288"

# Try to parse DJI metadata
dji_metadata1 = DJIMetadataParser(str(dataset1_dir))

# Create pipeline without GCPs first
pipeline1_no_gcp = OrthomosaicPipeline(
    image_dir=str(dataset1_dir),
    output_dir=str(output_dir / "dataset1_no_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=False,
    dji_metadata=dji_metadata1
)

print("Processing dataset 1 (without GCPs)...")
output1_no_gcp = pipeline1_no_gcp.run_full_pipeline(output_name="dataset1_no_gcp")


Parsing timestamp file: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_017_25-3288/DJI_202510060955_017_25-3288_Timestamp.MRK
Parsing navigation file: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_017_25-3288/DJI_202510060955_017_25-3288_PPKNAV.nav
Processing dataset 1 (without GCPs)...
Starting Orthomosaic Pipeline
Found 543 images


Loading images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [01:12<00:00,  7.48it/s]


Loaded 543 images
Loading features from cache: outputs/dataset1_no_gcp/cache/features.pkl
Loaded features for 543 images
Loading matches from cache: outputs/dataset1_no_gcp/cache/matches.pkl
Loaded 100 matches
Estimating initial camera poses...
Initialized poses for 2 cameras
Performing bundle adjustment...
Bundle adjustment: 1905 3D points, 3810 observations, 2 cameras
Initial mean reprojection error: 0.33 pixels
Note: Using simplified bundle adjustment. For production, consider OpenSfM or COLMAP.

Reprojection Errors:
  Mean: 0.33 pixels
  Median: 0.06 pixels
  Max: 382.70 pixels
Creating orthomosaic (resolution: 0.1m/pixel)...
Orthomosaic size: 1000x1004 pixels


Projecting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [00:01<00:00, 478.23it/s]

Orthomosaic saved to outputs/dataset1_no_gcp/dataset1_no_gcp.tif
Pipeline Complete


In [30]:
# Create pipeline with GCPs
pipeline1_with_gcp = OrthomosaicPipeline(
    image_dir=str(dataset1_dir),
    output_dir=str(output_dir / "dataset1_with_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=True,
    gcp_parser=gcp_parser,
    dji_metadata=dji_metadata1
)

print("Processing dataset 1 (with GCPs)...")
output1_with_gcp = pipeline1_with_gcp.run_full_pipeline(output_name="dataset1_with_gcp")


Processing dataset 1 (with GCPs)...
Starting Orthomosaic Pipeline
Found 543 images


Loading images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [01:14<00:00,  7.33it/s]


Loaded 543 images
Loading features from cache: outputs/dataset1_with_gcp/cache/features.pkl
Loaded features for 543 images
Loading matches from cache: outputs/dataset1_with_gcp/cache/matches.pkl
Loaded 100 matches
Estimating initial camera poses...
Initialized poses for 2 cameras
Performing bundle adjustment...
Bundle adjustment: 1905 3D points, 3810 observations, 2 cameras
Initial mean reprojection error: 0.33 pixels
Note: Using simplified bundle adjustment. For production, consider OpenSfM or COLMAP.

Reprojection Errors:
  Mean: 0.33 pixels
  Median: 0.06 pixels
  Max: 382.70 pixels
Creating orthomosaic (resolution: 0.1m/pixel)...
Orthomosaic size: 1000x1004 pixels


Projecting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 543/543 [00:01<00:00, 479.09it/s]

Orthomosaic saved to outputs/dataset1_with_gcp/dataset1_with_gcp.tif
Pipeline Complete


## 3. Visualize Results for Dataset 1


In [31]:
# Visualize feature matches
if len(pipeline1_no_gcp.matches) > 0:
    visualize_matches(
        pipeline1_no_gcp,
        match_idx=0,
        output_path=str(output_dir / "dataset1_match_example.png"),
        max_matches=100
    )

# Visualize reprojection errors
visualize_reprojection_errors(
    pipeline1_no_gcp,
    output_path=str(output_dir / "dataset1_reprojection_errors.png")
)

# Visualize camera poses
visualize_camera_poses(
    pipeline1_no_gcp,
    output_path=str(output_dir / "dataset1_camera_poses.png")
)

# Create match quality report
create_match_quality_report(
    pipeline1_no_gcp,
    output_dir=str(output_dir / "dataset1_matches")
)


Match visualization saved to outputs/dataset1_match_example.png
Reprojection error visualization saved to outputs/dataset1_reprojection_errors.png
Camera pose visualization saved to outputs/dataset1_camera_poses.png
Creating match quality report...
Match visualization saved to outputs/dataset1_matches/match_visualization_1.png
Match visualization saved to outputs/dataset1_matches/match_visualization_2.png
Match visualization saved to outputs/dataset1_matches/match_visualization_3.png
Match visualization saved to outputs/dataset1_matches/match_visualization_4.png
Match visualization saved to outputs/dataset1_matches/match_visualization_5.png
Match visualization saved to outputs/dataset1_matches/match_visualization_worst_96.png
Match visualization saved to outputs/dataset1_matches/match_visualization_worst_97.png
Match visualization saved to outputs/dataset1_matches/match_visualization_worst_98.png
Match visualization saved to outputs/dataset1_matches/match_visualization_worst_99.png
Mat

## 4. Process Second Dataset (DJI_202510060955_019_25-3288)


In [32]:
# Setup for second dataset
dataset2_dir = data_dir / "DJI_202510060955_019_25-3288"

# Try to parse DJI metadata
dji_metadata2 = DJIMetadataParser(str(dataset2_dir))

# Create pipeline without GCPs
pipeline2_no_gcp = OrthomosaicPipeline(
    image_dir=str(dataset2_dir),
    output_dir=str(output_dir / "dataset2_no_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=False,
    dji_metadata=dji_metadata2
)

print("Processing dataset 2 (without GCPs)...")
output2_no_gcp = pipeline2_no_gcp.run_full_pipeline(output_name="dataset2_no_gcp")


Parsing timestamp file: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_019_25-3288/DJI_202510060955_019_Timestamp.MRK
Parsing navigation file: /Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25/DJI_202510060955_019_25-3288/DJI_202510060955_019_PPKNAV.nav
Processing dataset 2 (without GCPs)...
Starting Orthomosaic Pipeline
Found 528 images


Loading images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [01:08<00:00,  7.66it/s]


Loaded 528 images
Loading features from cache: outputs/dataset2_no_gcp/cache/features.pkl
Loaded features for 528 images
Loading matches from cache: outputs/dataset2_no_gcp/cache/matches.pkl
Loaded 100 matches
Estimating initial camera poses...
Initialized poses for 2 cameras
Performing bundle adjustment...
Bundle adjustment: 10 3D points, 20 observations, 2 cameras
Initial mean reprojection error: 894.77 pixels
Note: Using simplified bundle adjustment. For production, consider OpenSfM or COLMAP.

Reprojection Errors:
  Mean: 894.77 pixels
  Median: 0.00 pixels
  Max: 8186.22 pixels
Creating orthomosaic (resolution: 0.1m/pixel)...
Orthomosaic size: 1000x1000 pixels


Projecting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [00:01<00:00, 436.73it/s]

Orthomosaic saved to outputs/dataset2_no_gcp/dataset2_no_gcp.tif
Pipeline Complete


In [33]:
# Create pipeline with GCPs
pipeline2_with_gcp = OrthomosaicPipeline(
    image_dir=str(dataset2_dir),
    output_dir=str(output_dir / "dataset2_with_gcp"),
    feature_detector="sift",
    max_features=5000,
    match_ratio=0.7,
    use_gcps=True,
    gcp_parser=gcp_parser,
    dji_metadata=dji_metadata2
)

print("Processing dataset 2 (with GCPs)...")
output2_with_gcp = pipeline2_with_gcp.run_full_pipeline(output_name="dataset2_with_gcp")


Processing dataset 2 (with GCPs)...
Starting Orthomosaic Pipeline
Found 528 images


Loading images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [01:08<00:00,  7.73it/s]


Loaded 528 images
Loading features from cache: outputs/dataset2_with_gcp/cache/features.pkl
Loaded features for 528 images
Loading matches from cache: outputs/dataset2_with_gcp/cache/matches.pkl
Loaded 100 matches
Estimating initial camera poses...
Initialized poses for 2 cameras
Performing bundle adjustment...
Bundle adjustment: 10 3D points, 20 observations, 2 cameras
Initial mean reprojection error: 894.77 pixels
Note: Using simplified bundle adjustment. For production, consider OpenSfM or COLMAP.

Reprojection Errors:
  Mean: 894.77 pixels
  Median: 0.00 pixels
  Max: 8186.22 pixels
Creating orthomosaic (resolution: 0.1m/pixel)...
Orthomosaic size: 1000x1000 pixels


Projecting images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 528/528 [00:01<00:00, 467.01it/s]

Orthomosaic saved to outputs/dataset2_with_gcp/dataset2_with_gcp.tif
Pipeline Complete


## 5. Download Basemap for Comparison


In [ ]:
# Convert UTM bounds to lat/lon for basemap download
import utm
from pathlib import Path

# Import GCPParser if not already imported
try:
    _ = GCPParser
except NameError:
    import sys
    sys.path.insert(0, str(Path.cwd()))
    from westminster_ground_truth_analysis import GCPParser

# Get GCP bounds if not already defined
try:
    # Check if bounds are already defined
    _ = min_x, min_y, max_x, max_y
    print("Using existing GCP bounds")
except NameError:
    # Get bounds from GCP parser (create parser if needed)
    try:
        _ = gcp_parser
    except NameError:
        # Create GCP parser if it doesn't exist
        # Check if data_dir is defined, if not use default path
        try:
            _ = data_dir
        except NameError:
            data_dir = Path("/Users/mauriciohessflores/Documents/Code/Data/New Westminster Oct _25")
            print(f"Using default data_dir: {data_dir}")
        
        gcp_file = data_dir / "25-3288-CONTROL-NAD83-UTM10N-EGM2008.csv"
        gcp_parser = GCPParser(str(gcp_file))
        print(f"Created GCP parser from {gcp_file}")
    
    # Get bounds from GCP parser
    min_x, min_y, max_x, max_y = gcp_parser.get_bounds()
    print(f"Retrieved GCP bounds: X=[{min_x:.2f}, {max_x:.2f}], Y=[{min_y:.2f}, {max_y:.2f}]")

# Convert GCP bounds from UTM to lat/lon
# UTM Zone 10N (based on GCP file name)
# NOTE: In the CSV, X column is actually Northing (~5.45M) and Y column is Easting (~500k)
# utm.to_latlon expects (easting, northing), so we need to swap them
center_easting = (min_y + max_y) / 2  # Y column is easting
center_northing = (min_x + max_x) / 2  # X column is northing

# Convert center point
lat_center, lon_center = utm.to_latlon(center_easting, center_northing, 10, 'N')

# Approximate bounds in lat/lon (rough conversion)
# For more accuracy, convert all corners
# Swap X and Y: Y is easting, X is northing
lat_min, lon_min = utm.to_latlon(min_y, min_x, 10, 'N')
lat_max, lon_max = utm.to_latlon(max_y, max_x, 10, 'N')

bbox = (min(lat_min, lat_max), min(lon_min, lon_max), 
        max(lat_min, lat_max), max(lon_min, lon_max))

print(f"Basemap bounding box: {bbox}")

# Download basemap
basemap_path = download_basemap(
    bbox=bbox,
    output_path=str(output_dir / "basemap.tif"),
    source="esri_world_imagery",
    target_resolution=0.1  # 0.1m per pixel
)


Basemap bounding box: (np.float64(49.20374809577927), np.float64(-122.90970020449743), np.float64(49.21168116673544), np.float64(-122.89956337139125))
Tile range: X [41571, 41579], Y [89799, 89790]


IndexError: list index out of range

## 6. Compare Orthomosaics to Basemap


In [ ]:
# Compare dataset 1 without GCPs
print("=" * 60)
print("Dataset 1 - Without GCPs")
print("=" * 60)
metrics1_no_gcp = compare_orthomosaic_to_basemap(
    str(output1_no_gcp),
    str(basemap_path),
    output_dir=str(output_dir / "comparison1_no_gcp")
)

# Compare dataset 1 with GCPs
print("\n" + "=" * 60)
print("Dataset 1 - With GCPs")
print("=" * 60)
metrics1_with_gcp = compare_orthomosaic_to_basemap(
    str(output1_with_gcp),
    str(basemap_path),
    output_dir=str(output_dir / "comparison1_with_gcp")
)


In [ ]:
# Compare dataset 2 without GCPs
print("=" * 60)
print("Dataset 2 - Without GCPs")
print("=" * 60)
metrics2_no_gcp = compare_orthomosaic_to_basemap(
    str(output2_no_gcp),
    str(basemap_path),
    output_dir=str(output_dir / "comparison2_no_gcp")
)

# Compare dataset 2 with GCPs
print("\n" + "=" * 60)
print("Dataset 2 - With GCPs")
print("=" * 60)
metrics2_with_gcp = compare_orthomosaic_to_basemap(
    str(output2_with_gcp),
    str(basemap_path),
    output_dir=str(output_dir / "comparison2_with_gcp")
)


## 7. Summary and Comparison


In [ ]:
# Create comparison summary
import pandas as pd

summary_data = {
    'Dataset': ['Dataset 1', 'Dataset 1', 'Dataset 2', 'Dataset 2'],
    'GCPs Used': ['No', 'Yes', 'No', 'Yes'],
    'RMSE': [
        metrics1_no_gcp['rmse'],
        metrics1_with_gcp['rmse'],
        metrics2_no_gcp['rmse'],
        metrics2_with_gcp['rmse']
    ],
    'MAE': [
        metrics1_no_gcp['mae'],
        metrics1_with_gcp['mae'],
        metrics2_no_gcp['mae'],
        metrics2_with_gcp['mae']
    ],
    'Correlation': [
        metrics1_no_gcp['correlation'],
        metrics1_with_gcp['correlation'],
        metrics2_no_gcp['correlation'],
        metrics2_with_gcp['correlation']
    ],
    'SSIM': [
        metrics1_no_gcp['ssim'],
        metrics1_with_gcp['ssim'],
        metrics2_no_gcp['ssim'],
        metrics2_with_gcp['ssim']
    ]
}

df = pd.DataFrame(summary_data)
print("\n" + "=" * 60)
print("SUMMARY COMPARISON")
print("=" * 60)
print(df.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics_to_plot = ['RMSE', 'MAE', 'Correlation', 'SSIM']
for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx // 2, idx % 2]
    x = np.arange(len(df))
    width = 0.35
    
    no_gcp_values = df[df['GCPs Used'] == 'No'][metric].values
    with_gcp_values = df[df['GCPs Used'] == 'Yes'][metric].values
    
    ax.bar(x - width/2, no_gcp_values, width, label='Without GCPs', alpha=0.7)
    ax.bar(x + width/2, with_gcp_values, width, label='With GCPs', alpha=0.7)
    
    ax.set_xlabel('Dataset')
    ax.set_ylabel(metric)
    ax.set_title(f'{metric} Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(['Dataset 1', 'Dataset 2'])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(output_dir / "accuracy_comparison.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\nComparison visualization saved to {output_dir / 'accuracy_comparison.png'}")
